# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 8:30AM,249571,10,8700067,Eywa Pharma Inc.,Released
1,Oct 27 2022 4:19PM,249572,15,PNC391151A,"Person & Covey, Inc.",Released
2,Oct 27 2022 4:19PM,249572,15,PNC391527A,"Person & Covey, Inc.",Released
3,Oct 27 2022 4:19PM,249572,15,PNC391528A,"Person & Covey, Inc.",Released
4,Oct 27 2022 4:19PM,249572,15,PNC391530A,"Person & Covey, Inc.",Released
5,Oct 27 2022 4:19PM,249572,15,PNC391531A,"Person & Covey, Inc.",Released
6,Oct 27 2022 4:19PM,249572,15,PNC391534A,"Person & Covey, Inc.",Released
7,Oct 27 2022 4:19PM,249572,15,PNC394411A,"Person & Covey, Inc.",Released
8,Oct 27 2022 4:19PM,249572,15,PNC394441A,"Person & Covey, Inc.",Released
9,Oct 27 2022 4:19PM,249572,15,PNC394477A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,249567,Released,3
20,249569,Executing,2
21,249570,Released,1
22,249571,Released,1
23,249572,Released,28


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249567,NaN,NaN,3.0
249569,NaN,2.0,NaN
249570,NaN,NaN,1.0
249571,NaN,NaN,1.0
249572,NaN,NaN,28.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249479,0.0,1.0,0.0
249485,0.0,1.0,0.0
249494,0.0,1.0,0.0
249516,0.0,20.0,7.0
249519,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249479,0,1,0
249485,0,1,0
249494,0,1,0
249516,0,20,7
249519,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249479,0,1,0
1,249485,0,1,0
2,249494,0,1,0
3,249516,0,20,7
4,249519,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249479,,1,
1,249485,,1,
2,249494,,1,
3,249516,,20,7
4,249519,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc."
29,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.
30,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc."
32,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc."
35,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc."
36,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation
49,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc."
50,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc."
51,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.,,,1
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",,,28
2,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,,,1
3,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc.",,2,
4,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",,,3
5,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",,,1
6,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,3,10,
7,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",,,1
8,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",,,1
9,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.,1,,
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28,,
2,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,1,,
3,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc.",,2,
4,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3,,
5,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1,,
6,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,,10,3
7,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",1,,
8,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",1,,
9,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.,1,,
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28,,
2,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,1,,
3,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc.",,2,
4,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28.0,NaN,NaN
2,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
3,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc.",NaN,2.0,NaN
4,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 8:30AM,249571,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Oct 27 2022 4:19PM,249572,15,"Person & Covey, Inc.",28.0,0.0,0.0
2,Oct 27 2022 4:07PM,249570,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
3,Oct 27 2022 3:42PM,249569,19,"GCH Granules USA, Inc.",0.0,2.0,0.0
4,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,499129,1.0,10.0,3.0
15,998162,110.0,20.0,0.0
16,1247609,4.0,1.0,0.0
19,249569,0.0,2.0,0.0
20,748546,3.0,2.0,0.0
21,1746797,6.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,499129,1.0,10.0,3.0
1,15,998162,110.0,20.0,0.0
2,16,1247609,4.0,1.0,0.0
3,19,249569,0.0,2.0,0.0
4,20,748546,3.0,2.0,0.0
5,21,1746797,6.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1.0,10.0,3.0
1,15,110.0,20.0,0.0
2,16,4.0,1.0,0.0
3,19,0.0,2.0,0.0
4,20,3.0,2.0,0.0
5,21,6.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1.0
1,15,Released,110.0
2,16,Released,4.0
3,19,Released,0.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,20.0,1.0,2.0,2.0,1.0
Released,1.0,110.0,4.0,0.0,3.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,20.0,1.0,2.0,2.0,1.0
2,Released,1.0,110.0,4.0,0.0,3.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,20.0,1.0,2.0,2.0,1.0
2,Released,1.0,110.0,4.0,0.0,3.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()